In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
movie = pd.read_csv(r"C:\Users\DIGIMAHAN\Desktop\dataset csv\movies.csv")
movie.sample(4)

,movieId,title,genres
21056,102415,Hyenas (Hyènes) (1992),Comedy|Drama
27627,128203,"Lightning, the White Stallion (1986)",Adventure|Children
21273,103255,Dirty Wars (2013),Documentary|War
17896,89664,Photos in the City of Sylvia (Unas fotos en la...,Documentary


In [3]:
rating = pd.read_csv(r"C:\Users\DIGIMAHAN\Desktop\dataset csv\ratings.csv")
rating.sample(3)

,userId,movieId,rating,timestamp
2479098,26780,466,4.0,1284984352
1628058,17383,475,3.5,1334799962
1249897,13329,3617,3.0,981132683


In [4]:
movie['year']=movie.title.str.extract('(\(\d\d\d\d\))')
movie['year']=movie.year.str.extract('(\d\d\d\d)')
movie['title']=movie.title.str.replace('(\(\d\d\d\d\))','',regex=True)
movie['title']=movie['title'].apply(lambda x:x.strip())

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\DIGIMAHAN\AppData\Local\Temp\ipykernel_6060\2242016959.py:1: SyntaxWarning: invalid escape sequence '\('
  movie['year']=movie.title.str.extract('(\(\d\d\d\d\))')
C:\Users\DIGIMAHAN\AppData\Local\Temp\ipykernel_6060\2242016959.py:2: SyntaxWarning: invalid escape sequence '\d'
  movie['year']=movie.year.str.extract('(\d\d\d\d)')
C:\Users\DIGIMAHAN\AppData\Local\Temp\ipykernel_6060\2242016959.py:3: SyntaxWarning: invalid escape sequence '\('
  movie['title']=movie.title.str.replace('(\(\d\d\d\d\))','',regex=True)


In [5]:
movie = movie.drop('genres',axis=1)      
movie.head()                                                          

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [6]:
rating = rating.drop('timestamp',axis=1)
rating.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [7]:
userinput=[{"title":'Breakfast Club, The','rating':5},
            {"title":'Toy Story','rating':3.5},
            {"title":'Jumanji','rating':2},
            {"title":'Pulp Fiction','rating':5},
            {"title":'Now and Then','rating':4.5}]

In [8]:
inputmovies=pd.DataFrame(userinput)
inputmovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Now and Then,4.5


In [9]:
inputid=movie[movie["title"].isin(inputmovies["title"].tolist())]
inputmovies=pd.merge(inputid,inputmovies)
inputmovies=inputmovies.drop("year",axis=1)
inputmovies.head()

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,27,Now and Then,4.5
3,296,Pulp Fiction,5.0
4,1968,"Breakfast Club, The",5.0


In [10]:
usersubset=rating[rating['movieId'].isin(inputmovies["movieId"].tolist())]
usersubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
681,14,296,2.0
749,15,1,4.0


In [11]:
usersubsetgroup =usersubset.groupby(["userId"])
usersubsetgroup.get_group(1130)

C:\Users\DIGIMAHAN\AppData\Local\Temp\ipykernel_6060\3321009229.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  usersubsetgroup.get_group(1130)


,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104443,1130,1968,4.5


In [12]:
usersubsetgroup=sorted(usersubsetgroup,key=lambda x:len(x[1]),reverse=True)

In [13]:
usersubsetgroup[0:100]

[((373,),
         userId  movieId  rating
  31219     373        1     4.0
  31220     373        2     1.5
  31233     373       27     2.0
  31298     373      296     4.5
  31629     373     1968     3.0),
 ((815,),
         userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73766     815       27     2.5
  73922     815      296     5.0
  74678     815     1968     4.5),
 ((2204,),
          userId  movieId  rating
  198937    2204        1     5.0
  198938    2204        2     3.0
  198949    2204       27     3.0
  198998    2204      296     5.0
  199222    2204     1968     4.0),
 ((2596,),
          userId  movieId  rating
  237718    2596        1     3.5
  237719    2596        2     2.5
  237726    2596       27     3.5
  237777    2596      296     4.5
  238028    2596     1968     3.5),
 ((4208,),
          userId  movieId  rating
  387004    4208        1     4.0
  387005    4208        2     2.0
  387022    4208       27     3.0

In [14]:
from math import sqrt

In [34]:
pearsoncor ={}
for name,group in usersubsetgroup:
    if isinstance(name, tuple):
        name = name[0]
    group=group.sort_values(by='movieId')
    nratings =len(group)
    temp_df=inputmovies[inputmovies['movieId'].isin(group["movieId"].tolist())]
    tempratinglist=temp_df["rating"].tolist()
    tempgrouplist=group["rating"].tolist()
    sxx=sum([i**2 for i in tempratinglist])-pow(sum(tempratinglist),2)/float(nratings)
    syy=sum([i**2 for i in tempgrouplist])-pow(sum(tempgrouplist),2)/float(nratings)
    sxy=sum(i*j for i,j in zip(tempratinglist,tempgrouplist))-sum(tempratinglist)*sum(tempgrouplist)/float(nratings)
    if sxx!=0 and syy!=0:
        pearsoncor[name]=sxy/sqrt(sxx*syy)
    else:
        pearsoncor[name]=0

In [39]:
pearsondf = pd.DataFrame.from_dict(pearsoncor,orient='index')
pearsondf.columns=["similarity"]
pearsondf["userId"]=pearsondf.index
pearsondf.index = range(len(pearsondf))
pearsondf.head()

,similarity,userId
0,0.538462,373
1,0.452308,815
2,0.392232,2204
3,0.832050,2596
4,0.294174,4208


In [40]:
topuser = pearsondf.sort_values(by='similarity',ascending=False)[0:50]
topuser

,similarity,userId
1369,1.0,8051
3172,1.0,39386
1725,1.0,13898
1694,1.0,13376
3077,1.0,37463
1579,1.0,11596
1510,1.0,10372
2984,1.0,36011
3324,1.0,42025
1881,1.0,16326


In [18]:
print(topuser['userId'].dtype)
print(rating['userId'].dtype)

print(topuser['userId'].head())
print(rating['userId'].head())


object
int64
1369     (8051,)
3172    (39386,)
1725    (13898,)
1694    (13376,)
3077    (37463,)
Name: userId, dtype: object
0    1
1    1
2    1
3    2
4    2
Name: userId, dtype: int64


In [19]:
topuser['userId'] = topuser['userId'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
topuser['userId'] = topuser['userId'].astype(int)


In [20]:
topuserrating=topuser.merge(rating,left_on="userId",right_on='userId',how='inner')
topuserrating.head()

,similarity,userId,movieId,rating
0,1.0,8051,1,3.5
1,1.0,8051,4,2.0
2,1.0,8051,5,1.0
3,1.0,8051,16,3.5
4,1.0,8051,32,3.0


In [21]:
topuserrating["weightedrating"]=topuserrating["similarity"]*topuserrating["rating"]
topuserrating.head()

,similarity,userId,movieId,rating,weightedrating
0,1.0,8051,1,3.5,3.5
1,1.0,8051,4,2.0,2.0
2,1.0,8051,5,1.0,1.0
3,1.0,8051,16,3.5,3.5
4,1.0,8051,32,3.0,3.0


In [22]:
topuserrating=topuserrating.groupby('movieId').sum()[["similarity",'weightedrating']]
topuserrating.columns=["sum_similarityindex",'sum_weightedrating']
topuserrating.head()

,sum_similarityindex,sum_weightedrating
movieId,,
1,43.0,147.5
2,7.0,21.0
3,6.0,15.5
4,4.0,6.5
5,9.0,21.5


In [23]:
recommendation=pd.DataFrame()
recommendation["weighted average recommendation score"]=topuserrating["sum_weightedrating"]/topuserrating["sum_similarityindex"]
recommendation["movieId"]=topuserrating.index
recommendation.head()

,weighted average recommendation score,movieId
movieId,,
1,3.430233,1
2,3.000000,2
3,2.583333,3
4,1.625000,4
5,2.388889,5


In [24]:
recommendation=recommendation.sort_values(by='weighted average recommendation score',ascending=False)
recommendation.head(10)

,weighted average recommendation score,movieId
movieId,,
1112,5.0,1112
932,5.0,932
3618,5.0,3618
3544,5.0,3544
3549,5.0,3549
45186,5.0,45186
5325,5.0,5325
927,5.0,927
929,5.0,929


In [26]:
movie.loc[movie["movieId"].isin(recommendation.head(10)["movieId"].tolist())]


,movieId,title,year
910,927,"Women, The",1939
912,929,Foreign Correspondent,1940
915,932,"Affair to Remember, An",1957
1089,1112,Palookaville,1996
1165,1189,"Thin Blue Line, The",1988
3454,3544,Shakes the Clown,1992
3459,3549,Guys and Dolls,1955
3528,3618,Small Time Crooks,2000
5229,5325,Dogtown and Z-Boyz,2001
11009,45186,Mission: Impossible III,2006
